In [0]:
#Then you upload the files you already download to your drive
#Connect to your google drive

from google.colab import drive
drive.mount('/content/gdrive/')

In [0]:
#RUN
%pip install lycon
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
import lycon
import torchvision.datasets as dsets
from skimage import transform
import torchvision.transforms as transforms
import torchvision
from torch.autograd import Variable
import pandas as pd;
import numpy as np;
from torch.utils.data import Dataset, DataLoader
#from vis_utils import *
import random;
import math;
import matplotlib.pyplot as plt
import scipy.io as sio
import pandas as pd
from torchvision import models
from torch.optim import lr_scheduler
#from tensorboard_logger import configure, log_value
import time

In [0]:
#Read the images and ground_truth from drive

Fold_1 = sio.loadmat('/content/gdrive/My Drive/Licenta/Fold_1.mat')
Fold_2 = sio.loadmat('/content/gdrive/My Drive/Licenta/Fold_2.mat')
Fold_3 = sio.loadmat('/content/gdrive/My Drive/Licenta/Fold_3.mat')
Fold_4 = sio.loadmat('/content/gdrive/My Drive/Licenta/Fold_4.mat')
Fold_5 = sio.loadmat('/content/gdrive/My Drive/Licenta/Fold_5.mat')

fold_1 = Fold_1['Fold_1']
fold_2 = Fold_2['Fold_2']
fold_3 = Fold_3['Fold_3']
fold_4 = Fold_4['Fold_4']
fold_5 = Fold_5['Fold_5']


In [0]:
#RUN
def get_5_fold(index_4, index_1):
    x = [None]*5
    y = [None]*5
    t = [None]*5
    x[0] = fold_1[:,0]
    x[1] = fold_2[:,0]
    x[2] = fold_3[:,0]
    x[3] = fold_4[:,0]
    x[4] = fold_5[:,0]

    y[0] = fold_1[:,1:(512*512)+ 1]
    y[1] = fold_2[:,1:(512*512)+ 1]
    y[2] = fold_3[:,1:(512*512)+ 1]
    y[3] = fold_4[:,1:(512*512)+ 1]
    y[4] = fold_5[:,1:(512*512)+ 1]

    t[0] = fold_1[:,(512*512)+ 1:2*(512*512)+ 1]
    t[1] = fold_2[:,(512*512)+ 1:2*(512*512)+ 1]
    t[2] = fold_3[:,(512*512)+ 1:2*(512*512)+ 1]
    t[3] = fold_4[:,(512*512)+ 1:2*(512*512)+ 1]
    t[4] = fold_5[:,(512*512)+ 1:2*(512*512)+ 1]
    
    labels = np.concatenate([x[index_4[0]], x[index_4[1]], x[index_4[2]], x[index_4[3]], x[index_1[0]]], 0)
    images = np.concatenate([y[index_4[0]], y[index_4[1]], y[index_4[2]], y[index_4[3]], y[index_1[0]]], 0)
    masks = np.concatenate([t[index_4[0]], t[index_4[1]], t[index_4[2]], t[index_4[3]], t[index_1[0]]], 0)

    
    fold_first = x[index_4[0]].shape[0] + x[index_4[1]].shape[0]+  x[index_4[2]].shape[0] + x[index_4[3]].shape[0]
    fold_last = x[index_1[0]].shape[0]
    del x,y,t
    return labels, images, masks, fold_first, fold_last

In [0]:
#RUN

# Realize 5-fold cross-validation
from itertools import combinations 
  
# Get all combinations of [1, 2, 3, 4, 5] 
# and length 4
comb = combinations([0, 1, 2, 3, 4], 4) 
comb = list(comb)
#Choose a combination from 0..4
ind = 0
res = [ele  for ele in range(5) if ele   not in comb[ind]] 

#Print first 4-fold indexex for train, and 1-fold index for test
print('Indexes for train: ', comb[ind])
print('Index for test', res)

labels, images, masks, first_fold_nr, last_fold_nr = get_5_fold(comb[ind], res)
print(masks.shape, images.shape, first_fold_nr, last_fold_nr)

Indexes for train:  (0, 1, 2, 3)
Index for test [4]
(3049, 262144) (3049, 262144) 2406 643


In [0]:
#RUN
#free RAM, because session will crash
%reset_selective -f Fold_1
%reset_selective -f Fold_2
%reset_selective -f Fold_3
%reset_selective -f Fold_4
%reset_selective -f Fold_5

%reset_selective -f fold_1
%reset_selective -f fold_2
%reset_selective -f fold_3
%reset_selective -f fold_4
%reset_selective -f fold_5



In [0]:
#RUN
# Choose l1 and l2 depending on batch_size, and 5_fold generation
# batch should divide de size
# Here I hard coded the values for train and test, depending on folders that I have selected for train and test
l = [[2400, 630], [2430, 600], [2460, 570], [2370, 630], [2490, 540]]

size_train_loader = l[ind][0]
size_test_loader = l[ind][1]

labels = labels
images = images.reshape(3049, -1, 512)
masks = masks.reshape(3049, -1, 512)

print(labels.shape, images.shape, masks.shape)



(3049,) (3049, 512, 512) (3049, 512, 512)


In [0]:
# save to disk
PATH = "/content/gdrive/My Drive/Date_Medicale"
num_imgs = images.shape[0]
#for i in range(num_imgs):
    if i%200 == 0:
        print(i, num_imgs)

    shape = images[i].shape
    img = (images[i]/images[i].max()*255).astype(np.uint8)
    gt = (masks[i]*255).astype(np.uint8)
    img = np.repeat(img[:, :, None], 3, axis=2)
    
    img_path = "%s/Images/%04i.png" % (PATH, i)
    gt_path = "%s/GTs/%04i.png" % (PATH, i)
    lycon.save(img_path, img)
    lycon.save(gt_path, gt)
#     fig, (ax0, ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=4, figsize=(16, 7))
#     show_img(train_x[i], ax=ax0)
#     show_img(img, ax=ax1)
#     show_img(train_y[i], ax=ax2)
#     show_img(gt, ax=ax3)
#     plt.pause(0.0001)
    


## Data

In [0]:
PATH = "/content/gdrive/My Drive/Date_Medicale/"
load_name = '/data/DAVIS-2016/VladSegmDataset/models/un_model_128x128_v2'
save_name = '/data/DAVIS-2016/date_medicale/train/rmn_128x128_v5'

In [0]:
import glob
x_names = np.sort(np.array(glob.glob(PATH + "Images/*.png")))
y_names = np.sort(np.array(glob.glob(PATH + "GTs/*.png")))

In [0]:
x_names = np.sort(np.array(glob.glob(PATH + "Images/*.png")))
y_names = np.sort(np.array(glob.glob(PATH + "GTs/*.png")))

train_x = x_names[0: size_train_loader]
train_y = y_names[0: size_train_loader]

val_x = x_names[first_fold_nr: first_fold_nr + size_test_loader]
val_y = y_names[first_fold_nr: first_fold_nr + size_test_loader]

print(train_x.shape, train_y.shape)

(2400,) (2400,)


In [0]:
#Dataloader

class SegmentationDataset(Dataset):
    def __init__(self, x_paths, y_paths, tr=None):
        self.x_paths = x_paths 
        self.y_paths = y_paths
        self.tr = tr
        self.transform = torchvision.transforms.ToTensor()
        
    def transformm(self, image, mask):

        # Random horizontal flipping
        image = transforms.ToPILImage()(image)
        mask = transforms.ToPILImage()(mask)
        if random.random() > 0.5:
            image = TF.hflip(image)
            mask = TF.hflip(mask)

        # Random vertical flipping
        if random.random() > 0.5:
            image = TF.vflip(image)
            mask = TF.vflip(mask)
            
        if random.random() > 0.5:
            rotation = random.randint(15, 45)
            image = TF.rotate(image, rotation)
            mask = TF.rotate(mask, rotation)
            
        return image, mask
    def __getitem__(self, index):
        SIZE = 128
        image = lycon.load(self.x_paths[index])
        image = lycon.resize(image, width=SIZE, height=SIZE)
        
        mask = lycon.load(self.y_paths[index])
        mask = lycon.resize(mask, width=SIZE, height=SIZE)
        mask = mask[:, :, 0]
        
        if self.tr == 1:
          image, mask = self.transformm(image, mask)
        # Transform to tensor
        image = self.transform(image)
        mask = self.transform(mask)
        return image, mask

    def __len__(self):
        return len(self.x_paths)

In [0]:
def show_img(im, figsize=None, ax=None, alpha=None):
    if not ax: fig,ax = plt.subplots(figsize=figsize)
    ax.imshow(im, alpha=alpha)
#     ax.set_axis_off()
    ax.grid()
    return ax

In [0]:
#Arhitecture of segmentation model, VGG encoder

import torch, numpy as np, os.path as osp, torch.nn as nn

def cached_download(url, path, md5=None, quiet=False, postprocess=None):

    def check_md5(path, md5):
        print(('[{:s}] Checking md5 ({:s})').format(path, md5))
        return md5sum(path) == md5

    if osp.exists(path):
        if not md5:
            print(('[{:s}] File exists ({:s})').format(path, md5sum(path)))
        if osp.exists(path):
            if md5:
                if check_md5(path, md5):
                    pass
                dirpath = osp.dirname(path)
                if not osp.exists(dirpath):
                    os.makedirs(dirpath)
                gdown.download(url, path, quiet=quiet)
        if postprocess is not None:
            postprocess(path)
        return path


def get_upsampling_weight(in_channels, out_channels, kernel_size):
    """Make a 2D bilinear kernel suitable for upsampling"""
    factor = (kernel_size + 1) // 2
    if kernel_size % 2 == 1:
        center = factor - 1
    else:
        center = factor - 0.5
    og = np.ogrid[:kernel_size, :kernel_size]
    filt = (1 - abs(og[0] - center) / factor) * (1 - abs(og[1] - center) / factor)
    weight = np.zeros((in_channels, out_channels, kernel_size, kernel_size), dtype=(np.float64))
    weight[range(in_channels), range(out_channels), :, :] = filt
    return torch.from_numpy(weight).float()


class FCN8s(nn.Module):
    pretrained_model = osp.expanduser('~/data/models/pytorch/fcn8s_from_caffe.pth')

    @classmethod
    def download(cls):
        return cached_download(url='http://drive.google.com/uc?id=0B9P1L--7Wd2vT0FtdThWREhjNkU',
          path=(cls.pretrained_model),
          md5='dbd9bbb3829a3184913bccc74373afbb')

    def __init__(self, n_class, vgg):
        super().__init__()
        self.conv1_1 = nn.Conv2d(3, 64, 3, padding=100)
        self.relu1_1 = nn.ReLU(inplace=True)
        self.conv1_2 = nn.Conv2d(64, 64, 3, padding=1)
        self.relu1_2 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(2, stride=2, ceil_mode=True)
        self.conv2_1 = nn.Conv2d(64, 128, 3, padding=1)
        self.relu2_1 = nn.ReLU(inplace=True)
        self.conv2_2 = nn.Conv2d(128, 128, 3, padding=1)
        self.relu2_2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(2, stride=2, ceil_mode=True)
        self.conv3_1 = nn.Conv2d(128, 256, 3, padding=1)
        self.relu3_1 = nn.ReLU(inplace=True)
        self.conv3_2 = nn.Conv2d(256, 256, 3, padding=1)
        self.relu3_2 = nn.ReLU(inplace=True)
        self.conv3_3 = nn.Conv2d(256, 256, 3, padding=1)
        self.relu3_3 = nn.ReLU(inplace=True)
        self.pool3 = nn.MaxPool2d(2, stride=2, ceil_mode=True)
        self.conv4_1 = nn.Conv2d(256, 512, 3, padding=1)
        self.relu4_1 = nn.ReLU(inplace=True)
        self.conv4_2 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu4_2 = nn.ReLU(inplace=True)
        self.conv4_3 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu4_3 = nn.ReLU(inplace=True)
        self.pool4 = nn.MaxPool2d(2, stride=2, ceil_mode=True)
        self.conv5_1 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu5_1 = nn.ReLU(inplace=True)
        self.conv5_2 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu5_2 = nn.ReLU(inplace=True)
        self.conv5_3 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu5_3 = nn.ReLU(inplace=True)
        self.pool5 = nn.MaxPool2d(2, stride=2, ceil_mode=True)
        self.fc6 = nn.Conv2d(512, 4096, 7)
        self.relu6 = nn.ReLU(inplace=True)
        self.drop6 = nn.Dropout2d()
        self.fc7 = nn.Conv2d(4096, 4096, 1)
        self.relu7 = nn.ReLU(inplace=True)
        self.drop7 = nn.Dropout2d()
        self.score_fr = nn.Conv2d(4096, n_class, 1)
        self.score_pool3 = nn.Conv2d(256, n_class, 1)
        self.score_pool4 = nn.Conv2d(512, n_class, 1)
        self.upscore2 = nn.ConvTranspose2d(n_class,
          n_class, 4, stride=2, bias=False)
        self.upscore8 = nn.ConvTranspose2d(n_class,
          n_class, 16, stride=8, bias=False)
        self.upscore_pool4 = nn.ConvTranspose2d(n_class,
          n_class, 4, stride=2, bias=False)
        self._initialize_weights()
        self.copy_params_from_vgg16(vgg)

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight.data.zero_()
                if m.bias is not None:
                    m.bias.data.zero_()
            if isinstance(m, nn.ConvTranspose2d):
                if not m.kernel_size[0] == m.kernel_size[1]:
                    raise AssertionError
                initial_weight = get_upsampling_weight(m.in_channels, m.out_channels, m.kernel_size[0])
                m.weight.data.copy_(initial_weight)

    def forward(self, x):
        h = x
        h = self.relu1_1(self.conv1_1(h))
        h = self.relu1_2(self.conv1_2(h))
        h = self.pool1(h)
        h = self.relu2_1(self.conv2_1(h))
        h = self.relu2_2(self.conv2_2(h))
        h = self.pool2(h)
        h = self.relu3_1(self.conv3_1(h))
        h = self.relu3_2(self.conv3_2(h))
        h = self.relu3_3(self.conv3_3(h))
        h = self.pool3(h)
        pool3 = h
        h = self.relu4_1(self.conv4_1(h))
        h = self.relu4_2(self.conv4_2(h))
        h = self.relu4_3(self.conv4_3(h))
        h = self.pool4(h)
        pool4 = h
        h = self.relu5_1(self.conv5_1(h))
        h = self.relu5_2(self.conv5_2(h))
        h = self.relu5_3(self.conv5_3(h))
        h = self.pool5(h)
        h = self.relu6(self.fc6(h))
        h = self.drop6(h)
        h = self.relu7(self.fc7(h))
        h = self.drop7(h)
        h = self.score_fr(h)
        h = self.upscore2(h)
        upscore2 = h
        h = self.score_pool4(pool4)
        h = h[:, :, 5:5 + upscore2.size()[2], 5:5 + upscore2.size()[3]]
        score_pool4c = h
        h = upscore2 + score_pool4c
        h = self.upscore_pool4(h)
        upscore_pool4 = h
        h = self.score_pool3(pool3)
        h = h[:, :,
         9:9 + upscore_pool4.size()[2],
         9:9 + upscore_pool4.size()[3]]
        score_pool3c = h
        h = upscore_pool4 + score_pool3c
        h = self.upscore8(h)
        h = h[:, :, 31:31 + x.size()[2], 31:31 + x.size()[3]].contiguous()
        return h

    def copy_params_from_vgg16(self, vgg16):
        features = [
         self.conv1_1, self.relu1_1,
         self.conv1_2, self.relu1_2,
         self.pool1,
         self.conv2_1, self.relu2_1,
         self.conv2_2, self.relu2_2,
         self.pool2,
         self.conv3_1, self.relu3_1,
         self.conv3_2, self.relu3_2,
         self.conv3_3, self.relu3_3,
         self.pool3,
         self.conv4_1, self.relu4_1,
         self.conv4_2, self.relu4_2,
         self.conv4_3, self.relu4_3,
         self.pool4,
         self.conv5_1, self.relu5_1,
         self.conv5_2, self.relu5_2,
         self.conv5_3, self.relu5_3,
         self.pool5]
        for l1, l2 in zip(vgg16.features, features):
            if isinstance(l1, nn.Conv2d) and isinstance(l2, nn.Conv2d):
                if not l1.weight.size() == l2.weight.size():
                    raise AssertionError
                if not l1.bias.size() == l2.bias.size():
                    raise AssertionError
                l2.weight.data.copy_(l1.weight.data)
                l2.bias.data.copy_(l1.bias.data)
                #l2.weight.requires_grad = False
                #l2.bias.requires_grad = False

        for i, name in zip([0, 3], ['fc6', 'fc7']):
            l1 = vgg16.classifier[i]
            l2 = getattr(self, name)
            l2.weight.data.copy_(l1.weight.data.view(l2.weight.size()))
            l2.bias.data.copy_(l1.bias.data.view(l2.bias.size()))
# okay decompiling fcn8s.cpython-36.pyc


In [0]:
trainset = SegmentationDataset(train_x, train_y, 1)
valset = SegmentationDataset(val_x, val_y)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=30, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(valset, batch_size=30, shuffle=False, num_workers=4)

test_loader = torch.utils.data.DataLoader(valset, batch_size=1, shuffle=False, num_workers=4)
trtest_loader =  torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=False, num_workers=4)

In [0]:
#RUN to test if data is normalized, and mask is correlated with image
#Printez sa vad daca s-a normalizat
#Print values from an image to see if values are normalized

t, l = trainset.__getitem__(2399)
print(l.shape)
print(t.shape)
#print(t[0][75])
print(trainset.__len__())
plt.figure(1)
plt.subplot(211)
plt.imshow(t.permute(1, 2, 0))

plt.subplot(212)
plt.imshow(l[0])
plt.show()


In [0]:
#Function to calculate dice between 2 images
def T_dice_coefficient(predicted, target):
        """Calculates the Sørensen–Dice Coefficient for a
        single sample.
        Parameters:
            predicted(numpy.ndarray): Predicted single output of the network.
                                    Shape - (Channel,Height,Width)
            target(numpy.ndarray): Actual required single output for the network
                                    Shape - (Channel,Height,Width)
        Returns:
            coefficient(float): Dice coefficient for the input sample.
                                        1 represents high similarity and
                                        0 represents low similarity.
        """
        smooth = 1
       # predicted = predicted.cpu().numpy()
        #target = target.cpu().numpy()
        product = np.multiply(predicted, target)
        intersection = np.sum(product)
        coefficient = (2*intersection + smooth) / \
            (np.sum(predicted) + np.sum(target) + smooth)
        return coefficient

#Accuracy metric function, dice score
def MeanDice(model, loader, len):
    total_score = 0
    model.eval()
    for i, (img, mask_t) in enumerate(loader):
        
        sizee = img.shape[0]
        if sizee != 1:
           raise Exception("Set batch size to 1 for testing purpose")
        else:
          
          data = img.to(device).float()
          with torch.no_grad():
             mask_pred = model(data)
         
          mask = mask_t[0][0]
          mask_pred = mask_pred[0][0]

          output = (mask_pred > 0.5)
          output = output.cpu().numpy()
          output = np.resize(output, (size, size))
          mask = np.resize(mask,(size, size))
          iou_score = T_dice_coefficient(output, mask)
          total_score += iou_score
    print(total_score)
    model.train()
    return total_score/len

In [0]:
#Accuracy metric function, IOU
def MeanIOU(model, loader, len, treshold = 0.5):
    total_score = 0
    for i, (img, mask_t) in enumerate(loader):
       
        sizee = img.shape[0]
        
        if sizee != 1:
           raise Exception("Set batch size to 1 for testing purpose")
        else:  
          data = img.to(device).float()
          with torch.no_grad():
             mask_pred = model(data)
         
          mask = mask_t[0][0]
          mask_pred = mask_pred[0][0]
          output = (mask_pred > treshold)
          output = output.cpu().numpy()
          output = np.resize(output, (size, size))
          intersection = np.logical_and(mask, output)
          union = np.logical_or(mask, output)
          iou_score = np.sum(intersection.cpu().detach().numpy()) / np.sum(union.cpu().detach().numpy())
          total_score += iou_score
    print(total_score)
    return total_score/len

In [0]:
#IOU function for binary-segmentation
def iou_cross(model, loader, len):
    total = 0
    to = 0
    for i, (img, mask_t) in enumerate(loader):
        
        data = img.expand(-1, 3, -1, -1)
        data = data.to(device).float()
        sizee = img.shape[0]
        data = img.to(device).float()

        if sizee != 1:
           raise Exception("Set batch size to 1 for testing purpose")
        else:
          with torch.no_grad(): 
            mask_pred = model(data)

        mask = mask_t[0]
        tt = mask_pred[0]
        lbl_pred = tt.max(0)[1].cpu()
        intersection = np.logical_and(mask, lbl_pred)
        union = np.logical_or(mask, lbl_pred)
        iou_score = np.sum(intersection.cpu().detach().numpy()) / np.sum(union.cpu().detach().numpy())
        total += iou_score
        
    return total/len

In [0]:
#dice function for binary-segmentation
def dice_cross(model, loader, len):
    total = 0
    for i, (img, mask_t) in enumerate(loader):
        
        data = img.expand(-1, 3, -1, -1)
        data = data.to(device).float()
        sizee = img.shape[0]
        data = img.to(device).float()

        if sizee != 1:
           raise Exception("Set batch size to 1 for testing purpose")
        else:
          with torch.no_grad(): 
            mask_pred = model(data)

        mask = mask_t[0]
        tt = mask_pred[0]
        lbl_pred = tt.max(0)[1].cpu()  
        
        iou_score = T_dice_coefficient(lbl_pred, mask)
        total += iou_score
        
    return total/len

In [0]:
#criterion = nn.BCEWithLogitsLoss()
n_classes = 2
lr = 1e-5
batch_size = 30
size = 128

#Load pretrained model
vgg = models.vgg16(pretrained=True)
fcn_model = FCN8s(n_classes, vgg).cuda()


In [0]:
#Set hyperparameters for training
epochs = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

lr = 0.000011
optimizer = torch.optim.Adam(fcn_model.parameters(), lr=lr)
criterion = nn.BCEWithLogitsLoss()
#scheduler  = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.75)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.85, patience=3, verbose=True)



In [0]:
torch.cuda.empty_cache()
print(optimizer)
print(criterion)

In [0]:
#Load model from path in drive

path = 'gdrive/My Drive/Licenta/FCN8s_dice/fcn_8s_B_dice.pt'
fcn_model.load_state_dict(torch.load(path))



In [0]:
#List of loses for plots

train_losses = []
test_losses = []
IOU = []
Dice = []
best_loss = 100000

#Path save
path_save = 'gdrive/My Drive/Licenta/FCN8s_dice/fcn_8s_B_diceee.pt'


In [0]:
#Train function

def train(best_loss, epochs):
    best = best_loss 
    for epoch in range(epochs):
        
        ts = time.time()
        fcn_model.train()
        loss_per_epoch = 0
        mean_loss = []
        
        for iter_no, (inputs, labels) in enumerate(train_loader):
            # train
            inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.expand(-1, 3, -1, -1)
            #labels = labels.long()
            #labels = labels.view(-1, size, size)
            
            optimizer.zero_grad()
            outputs = fcn_model(inputs)[:,:1]

            loss = criterion(outputs, labels)
            loss /= inputs.shape[0]
            loss.backward()
            optimizer.step()

            mean_loss.append(loss.detach().cpu())
            
            if iter_no%10 == 0:
                print ("epoch {}, iter {}, loss: {}".format(epoch, iter_no, loss))
        print('--------------END EPOCH--------------')
        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        print("Epoch", epoch, "mean_loss", np.mean(mean_loss))

        train_losses.append( np.mean(mean_loss))

        idx = np.random.randint(len(val_loader))
        fcn_model.eval()
        val_inputs, val_labels = valset[idx]
        print(idx)
        val_inputs, val_labels = val_inputs.cuda(), val_labels.cuda()
        val_outputs = fcn_model(val_inputs[None])[:, :1]
        show_img(val_inputs[0].data.cpu().numpy())
        plt.pause(0.00001)
        show_img(val_labels[0].data.cpu().numpy())
        plt.pause(0.00001)
        show_img(val_outputs[0, 0].data.cpu().numpy())
        plt.pause(0.00001)
        fcn_model.train()

        print('--------------Start VALID--------------')

        ep_loss = validate(fcn_model)
        scheduler.step(ep_loss)   
        ac = MeanIOU(fcn_model, test_loader, size_test_loader)
        dice = MeanDice(fcn_model, test_loader, size_test_loader)
        #ac = iou_cross(fcn_model, test_loader, size_test_loader)
        #dice = dice_cross(fcn_model, test_loader, size_test_loader)
        
        print('Accuracy:', ac)
        IOU.append(ac)
        Dice.append(dice)

        if ep_loss < best:
          print("Save model:")
          torch.save(fcn_model.state_dict(),path_save)
          best = ep_loss
    return best
    

In [0]:
#Validation function

def validate(model):
        
        fcn_model.eval()
        visualizations = []
        mean_loss = []
        model.eval()
        for iter, (inputs, labels) in enumerate(val_loader):
            
            optimizer.zero_grad()
            inputs = inputs.cuda()
            labels = labels.cuda()
            #labels = labels.long()
            inputs = inputs.expand(-1, 3, -1, -1)
            #labels = labels.view(-1, size, size)
            
            with torch.no_grad():
                score = model(inputs)[:,:1]
                       
            loss = criterion(score, labels)
            loss /= inputs.shape[0]
            mean_loss.append(loss.detach().cpu())

            if np.isnan(loss.detach().cpu()):
                raise ValueError('loss is nan while validating')
            if iter%10 == 0:
                print ("iter {}, loss: {}".format( iter, loss))
                
        print('Loss: ', np.mean(mean_loss))
        test_losses.append(np.mean(mean_loss))
        print('=================END_VALID===============')
        model.train()
        return np.mean(mean_loss)

In [0]:
epochs = 5
best = train(best_loss, epochs)
best_loss = best

In [0]:
#Plot & save plots function
from matplotlib import pyplot

def plot_loss(loss, label, name, xlabel, ylabel, color='blue'):
    pyplot.plot(loss, label=label, color=color)
    pyplot.xlabel(xlabel)
    pyplot.ylabel(ylabel)
    pyplot.legend()
    pyplot.savefig(name)
def save_plot(loss, label, name, color='blue'):
    pyplot.plot(loss, label=label, color=color)
    pyplot.legend()
    pyplot.savefig(name)
nume = 'FCN8s_bce)aug.png'
def plot_loss_t_t(loss1, loss2, label1, label2, name, color1='blue', color2='red'):
    pyplot.plot(loss1, label=label1, color=color1)
    pyplot.plot(loss2, label=label2, color=color2)
    pyplot.xlabel("Epoca")
    pyplot.ylabel("Eroare")
    pyplot.legend()
    pyplot.savefig(name)

In [0]:

plot_loss_t_t(train_losses, test_losses, 'train_loss', 'test_loss', '/content/gdrive/My Drive/Licenta/FCN8s_aug/train_test_loss' + nume,'red', 'blue')

In [0]:
plot_loss(IOU, 'IOU', '/content/gdrive/My Drive/Licenta/FCN8s_cross/mean_iou_' + nume,  'Epoca', 'Acuratete IOU', 'red')

In [0]:
plot_loss(Dice, 'Scor Dice', '/content/gdrive/My Drive/Licenta/FCN8s_cross/dice_' + nume,'Epoca', 'Scor Dice' ,'red')


In [0]:
#Evaluate 1 image

t = fcn_model.eval()
ind = 4                #Select the index of image from [0, batch_size - 1]

it = iter(val_loader)

img1, mask_t = next(it)
img = img1
print(img1.shape, mask_t.shape, img.shape)


img1 = img1.cuda()
mask_t = mask_t[ind].view(-1, 128)
mask_pred = fcn_model(img1)[:, :1]

mask_pred = mask_pred[ind]



print(mask_pred.shape)
output = (mask_pred > 0.5)
output = output.cpu().numpy()
output = np.resize(output, (128, 128))


t1 = mask_t[mask_t > 0 ].sum()
t2 = output[output > 0].sum()
print('----------------------------------------------------')
print("Nr Pixeli din tumoare: ", t1, "Pixeli prezisi: ", t2)  


f, axarr = plt.subplots(2,2)
axarr[0,0].imshow(img[ind][0].cpu())
axarr[0,1].imshow(mask_t)
axarr[1,0].imshow(img1[ind].cpu().permute(1, 2, 0), cmap = 'gray')
axarr[1,1].imshow(output)

In [0]:
#Function to calculate precision and recall for segmentation

def pixel_wise_boundary_precision_recall(pred, gt):
    """Evaluate voxel prediction accuracy against a ground truth.
    Parameters
    ----------
    pred : np.ndarray of int or bool, arbitrary shape
        The voxel-wise discrete prediction. 1 for boundary, 0 for non-boundary.
    gt : np.ndarray of int or bool, same shape as `pred`
        The ground truth boundary voxels. 1 for boundary, 0 for non-boundary.
    Returns
    -------
    pr : float
    rec : float
        The precision and recall values associated with the prediction.
    Notes
    -----
    Precision is defined as "True Positives / Total Positive Calls", and
    Recall is defined as "True Positives / Total Positives in Ground Truth".
    This function only calculates this value for discretized predictions,
    i.e. it does not work with continuous prediction confidence values.
    """
    tp = float((gt * pred).sum())
    fp = (pred * (1-gt)).sum()
    fn = (gt * (1-pred)).sum()
    if tp == 0:
      return 0, 0
    return tp/(tp+fp), tp/(tp+fn)


In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score, precision_recall_fscore_support

pred = torch.empty(1, 128, 128)
lab = torch.empty(1, 128, 128)

for i, (images, labels) in enumerate(test_loader):
          images = images.to(device).float()         # Convert torch tensor to Variable: change image from a vector of size 784 to a matrix of 28 x 28
          labels = labels.to(device).long()
         
          images = images.expand(-1, 3, -1, -1) #Because first layer of cnns have 3 channels for input and my image have 1 channel, i expand the input to 3 channels
          images = images.to(device).float()
         
          outputs = fcn_model(images)
           
          mask_pred = outputs[0][0]
          output = (mask_pred > 0.5)
          output = output.reshape(-1, 128, 128)
          labels = labels.view(-1, 128, 128)
          
          pred = torch.cat((pred, output.cpu().float()), 0)
          lab = torch.cat((lab, labels.cpu().float()), 0)


In [0]:
lab = lab[1:631]
pred = pred[1:631]
pr = 0
re = 0
for i in range(630):
  t1, t2 = pixel_wise_boundary_precision_recall(pred[i], lab[i])
  pr += t1
  re += t2
  
print(pr/630, re/630)

